In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('max_columns', None)

# IPFS

In [ ]:
df1 = pd.read_csv('1205_peer_ipfs.txt', sep=' ', header=None)
df1 = df1.rename(columns={0:"a",1:"b",2:"c"})
# get timestamp
df1['timestamp'] = df1['a'].map(str) +" "+ df1['b'].map(str)
df1['timestamp'] = df1['timestamp'].str.strip('[]')
df1['datetime'] = pd.to_datetime(df1['timestamp'])
# get peer
df1['peer'] = df1['c'].apply(lambda x: x.split('/')[-1])
df1['ip'] = df1['c'].apply(lambda x: x.split('/')[2])
df1 = df1.drop(['a','b','c','timestamp'],axis=1)
df1 = df1.drop_duplicates()
df1.shape

In [ ]:
df1.head()

In [ ]:
len(df1['ip'].unique())

In [ ]:
len(df1['peer'].unique())

In [ ]:
def getPivotDataframe(df):
    df1 = df.copy()
    df1['value'] = 1
    
    df2 = df1.pivot(index='datetime', columns='peer', values='value').T
    df2 = df2.fillna(0)
    # sort peers by order of occurance
    ll = list(df2.columns.strftime('%Y-%m-%d %H:%M:%S'))
    df2 = df2.sort_values(by=ll, axis=0, ascending=False)

    df2 = df2.reset_index()
    df2 = df2.drop(['peer'],axis=1)
    
    return df2

In [ ]:
df2 = getPivotDataframe(df1)
df2.head()

In [ ]:
fig = px.imshow(df2, color_continuous_scale="Cividis", origin='lower')
fig.update_xaxes(title="timestamp")
fig.update_yaxes(title="peer")
fig.show()

In [ ]:
df1_2 = df1[df1['datetime'].dt.strftime('%H:%M:%S').between('00:00:00','02:00:00')]

df2_2 = getPivotDataframe(df1_2)
df2_2.shape

In [ ]:
fig = px.imshow(df2_2, color_continuous_scale="Cividis", origin='lower')
fig.update_xaxes(title="timestamp")
fig.update_yaxes(title="peer")
fig.show()

# Swarm

In [ ]:
df3 = pd.read_csv('1205_peer_swarm.txt', sep=' ', header=None)
df3 = df3.dropna()

df4 = pd.DataFrame(columns=['timestamp', 'peer'])

def func(a, i):
    a[i] = a[i].split('"')[3]

for index, row in df3.iterrows():
    timestamp = (row[0]+" "+row[1]).strip('[]')
    strs = row[2].split('{')[2:]
    list(map(lambda i:func(strs, i), range(0, len(strs))))
    df_temp = pd.DataFrame(strs)
    df_temp = df_temp.rename(columns={0: "peer"})
    df_temp['timestamp'] = timestamp
    df4 = pd.concat([df_temp, df4])

df4['datetime'] = pd.to_datetime(df4['timestamp'])
df4 = df4.drop(['timestamp'],axis=1)
df4.shape

In [ ]:
df4.head()

In [ ]:
len(df4['peer'].unique())

In [ ]:
df5 = getPivotDataframe(df4)
df5.head()

In [ ]:
fig = px.imshow(df5, color_continuous_scale="Cividis", origin='lower')
fig.update_xaxes(title="timestamp")
fig.update_yaxes(title="peer")
fig.show()

In [ ]:
df4_2 = df4[df4['datetime'].dt.strftime('%H:%M:%S').between('00:00:00','02:00:00')]

df5_2 = getPivotDataframe(df4_2)
df5_2.shape

In [ ]:
fig = px.imshow(df5_2, color_continuous_scale="Cividis", origin='lower')
fig.update_xaxes(title="timestamp")
fig.update_yaxes(title="peer")
fig.show()

# Compare

In [ ]:
df6 = df1[['peer','datetime']].groupby('datetime').agg('count')
df6 = df6.rename(columns={'peer':'count'})
df6 = df6.reset_index()
df6['node'] = 'ipfs'

In [ ]:
df6.describe()

In [ ]:
df7 = df4[['peer','datetime']].groupby('datetime').agg('count')
df7 = df7.rename(columns={'peer':'count'})
df7 = df7.reset_index()
df7['node'] = 'swarm'

In [ ]:
df7.describe()

In [ ]:
df8 = pd.concat([df6, df7])
df8.head()

In [ ]:
fig = px.line(df8, x='datetime', y='count', color='node')
fig.update_yaxes(title="number of connected peers")
fig.show()